In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

setores = ["Alimentação", "Casa e Construção", "Comunicação, Informática e Eletrônicos", "Educação", 
            "Entretenimento e Lazer", "Hotelaria e Turismo", "Limpeza e Conservação", "Moda", "Saúde, Beleza e Bem Estar",
            "Serviços Automotivos", "Serviços e Outros Negócios", "MicroFranquias", "Home Based"]
urls = {
    "Alimentação": 'https://franquias.portaldofranchising.com.br/franquias-de-alimentacao/?_gl=1*1dldi9q*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDM1MTM1Ni40LjEuMTcwMDM1MTM2Mi41NC4wLjA.',
    "Casa e Construção" : 'https://franquias.portaldofranchising.com.br/franquias-de-casa-e-construcao/?_gl=1*1fwi1bb*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxODk4OC4yNi4wLjA.',
    "Comunicação, Informática e Eletrônicos": 'https://franquias.portaldofranchising.com.br/franquias-de-comunicacao-informatica-e-eletronicos/?_gl=1*j3armv*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTM5NC40MC4wLjA.',
    "Educação": 'https://franquias.portaldofranchising.com.br/franquias-de-servicos-educacionais/?_gl=1*bmpnh6*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTQzNC42MC4wLjA.',
    "Entretenimento e Lazer": 'https://franquias.portaldofranchising.com.br/franquias-de-entretenimento-e-lazer/?_gl=1*1p3jnjm*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTUwMi41Ni4wLjA.',
    "Hotelaria e Turismo": 'https://franquias.portaldofranchising.com.br/franquias-de-hotelaria-e-turismo/?_gl=1*1wajsbn*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTU2OS41Ni4wLjA.',
    "Limpeza e Conservação": 'https://franquias.portaldofranchising.com.br/franquias-de-limpeza-e-conservacao/?_gl=1*xqtwk5*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTYzMi41Ni4wLjA.',
    "Moda": 'https://franquias.portaldofranchising.com.br/franquias-de-moda/?_gl=1*y9dy57*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTY3MS4xNy4wLjA.',
    "Saúde, Beleza e Bem Estar": 'https://franquias.portaldofranchising.com.br/franquias-de-saude-beleza-e-bem-estar/?_gl=1*kkua6c*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTcyNC41Ny4wLjA.',
    "Serviços Automotivos": 'https://franquias.portaldofranchising.com.br/franquias-de-servicos-automotivos/?_gl=1*1ncvquy*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTc4NS41Ni4wLjA.',
    "Serviços e Outros Negócios": 'https://franquias.portaldofranchising.com.br/franquias-de-servicos-e-outros-negocios/?_gl=1*ealeka*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTgzMi45LjAuMA..',
    "MicroFranquias": 'https://franquias.portaldofranchising.com.br/microfranquias/?_gl=1*91qu51*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUxOTk3NC4yNi4wLjA.',
    "Home Based": 'https://franquias.portaldofranchising.com.br/franquias-home-based/?_gl=1*9g68sp*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDUxODk1NC44LjEuMTcwMDUyMDA1My40My4wLjA.'


}

colunas = ['Setor','Nome', 'Unidades', 'Investimento', 'Retorno', 'Faturamento', 'Contato', 'Referência']
faturamento = "Não informado"

df = pd.DataFrame(columns=colunas)

for setor in setores:
    url = urls[setor]

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    filtro1 = soup.find_all('a')
    for e in filtro1:
        if e.get('title'):
            if not ('#' in e.get('href')):
                nome = e.get('title')
                url_franquia = e.get('href')
                page_franquia = requests.get(url_franquia)
                soup = BeautifulSoup(page_franquia.text, 'html.parser')

                achou_unidades = False
                achou_investimento = False
                achou_retorno = False
                achou_contato = False
                achou_faturamento = False

                for e in soup.find_all('strong'):
                    conteudo = e.get_text()
                    if len(conteudo)>0:
                        separacao_string = conteudo.split()
                    else:
                        separacao_string=['N']
                        
                    if not achou_investimento and separacao_string[0] == 'a':
                        investimento = ' '.join(separacao_string)
                        achou_investimento = True
                    elif not achou_retorno and separacao_string[0] == 'de':
                        retorno = ' '.join(separacao_string)
                        achou_retorno = True
                    elif not achou_faturamento and separacao_string[0][0] == 'R':
                        faturamento = ' '.join(separacao_string)
                        achou_faturamento = True
                    elif not achou_contato and len(separacao_string)==2:
                        contato = ' '.join(separacao_string)
                        achou_contato = True
                    elif not achou_unidades:
                        n_unidades = int(' '.join(separacao_string))
                        achou_unidades = True
                
                dados = {
                    'Setor':setor,
                    'Nome':nome,
                    'Unidades':n_unidades,
                    'Investimento':investimento,
                    'Retorno':retorno,
                    'Faturamento':faturamento,
                    'Contato':contato,
                    'Referência':url_franquia
                }

                if nome not in df['Nome'].values:
                    df = df._append(dados, ignore_index=True)


# Ordena por franquias com maior quantidade de unidades
df_ordenado = df.sort_values(by="Unidades", ascending=False)
maior_unidade = df_ordenado['Unidades'].iloc[0]
referencia = df.loc[df['Unidades'] == maior_unidade, 'Referência'].values[0]

print(f'Franquia com maior quantidade de unidades {df_ordenado["Nome"].iloc[0]}; Referência {referencia}')

display(df_ordenado.head(30))


Franquia com maior quantidade de unidades CACAU SHOW; Referência https://franquias.portaldofranchising.com.br/franquia-cacau-show-valor/


,Setor,Nome,Unidades,Investimento,Retorno,Faturamento,Contato,Referência
68,Alimentação,CACAU SHOW,3868,a partir de R$64.900,de 14 a 24 meses,R$40.000,(11) 99737-7777,https://franquias.portaldofranchising.com.br/f...
90,Casa e Construção,COLCHÕES ORTOBOM,2373,a partir de R$100.000,de 12 a 18 meses,R$80.000,(21) 2197-1000,https://franquias.portaldofranchising.com.br/f...
250,Serviços e Outros Negócios,MARKET4U,2100,a partir de R$60.000,de 12 a 25 meses,R$35.000,(41) 99194-2636,https://franquias.portaldofranchising.com.br/f...
191,"Saúde, Beleza e Bem Estar",ODONTOCOMPANY,2013,a partir de R$189.010,de 24 a 36 meses,R$148.500,(11) 3164-9779,https://franquias.portaldofranchising.com.br/f...
276,Serviços e Outros Negócios,SEGURALTA - BOLSA DE SEGUROS,1755,a partir de R$32.500,de 12 a 36 meses,R$30.000,(17) 3214-9300,https://franquias.portaldofranchising.com.br/f...
3,Alimentação,SUBWAY,1574,a partir de R$490.000,de 36 a 60 meses,R$42.000,(11) 3077-1010,https://franquias.portaldofranchising.com.br/f...
215,"Saúde, Beleza e Bem Estar",ÓTICAS CAROL,1460,a partir de R$320.000,de 24 a 48 meses,R$83.000,(11) 2395-5194,https://franquias.portaldofranchising.com.br/f...
220,"Saúde, Beleza e Bem Estar",ÓTICAS DINIZ,1206,a partir de R$260.000,de 15 a 18 meses,R$200.000,(11) 2249-8181,https://franquias.portaldofranchising.com.br/f...
110,Educação,WIZARD BY PEARSON,1024,a partir de R$100.000,de 18 a 24 meses,R$110.000,(19) 99966-4313,https://franquias.portaldofranchising.com.br/f...
62,Alimentação,OGGI SORVETES,1016,a partir de R$50.000,de 18 a 24 meses,"REI DO MATE,",(11) 4398-6830,https://franquias.portaldofranchising.com.br/f...


ModuleNotFoundError: No module named 'openpyxl'

In [2]:
# Especificar o caminho do arquivo Excel
caminho_arquivo_excel = 'franquias.xlsx'

# Salvar o dataframe no arquivo Excel
df_ordenado.to_excel(caminho_arquivo_excel, index=False)